# Grove PH Sensor Kit

This example shows how to use the [Grove PH Sensor Kit](https://wiki.seeedstudio.com/Grove-PH-Sensor-kit/). The Grove PH sensor produces an analog signal, and requires an ADC. 

A Grove PH sensor and Pynq Grove Adapter, or Pynq Shield is required. The Grove PH Sensor, Pynq Grove Adapter, and Grove I2C ADC are used for this example. 

Every PH sensor kit requires calibration before use. Before detecting the target solution, the sensor MUST be calibrated by pointed calibration fluid, and it also MUST be put into pointed buffer(PH=7) or clean water before detecting a new kind of solution and swiped.

It takes about 30 to 60 seconds for the PH reading to stabilize.

In [ ]:
# Initialize

from pynq.overlays.base import BaseOverlay
from pynq.lib.pmod import Grove_ADC
from pynq.lib.pmod import PMOD_GROVE_G4 

base = BaseOverlay("base.bit")

#### For this example, we use PMODB and GROVE G4 on the Grove Adapter

In [ ]:
grove_adc = Grove_ADC(base.PMODB,PMOD_GROVE_G4)
print("voltage : {} V".format(round(grove_adc.read(),4)))

In [ ]:
# Utility functions
# https://wiki.seeedstudio.com/Grove-PH-Sensor-kit/

def average_data(arr) :
  amount = 0
  n = len(arr)

  if (n < 5) :
    return sum(arr) / n
  else :
    if (arr[0] < arr[1]) :
      min = arr[0]
      max = arr[1]
    else :
      min = arr[1]
      max = arr[0]
    for i in range (2,n) :
      if (arr[i] < min) :
        amount += min
        min = arr[i]
      elif (arr[i] > max) : 
          amount += max
          max = arr[i]
      else :
        amount += arr[i]
    avg = amount / (n - 2)
  return avg

def get_avg_volts(interval, count):
    samples= []
    for i in range(0, count):
        samples.append(round(grove_adc.read(),10))
    #print(samples)
    return average_data(samples)

### Calibration Solution 1

Remove the PH sensor, clean it with fresh water and insert in first calibration solution.

#### Update variable "PH1" to the PH of reference solution 1

Wait for 30-60 seconds and run the cell below.
It is recommended to run the cell multiple times until V1 value stabilizes.

#### IMPORTANT : Run this cell only while calibrating the sensor

In [ ]:
# IMPORTANT : Run this cell only while calibrating the sensor

# Calibration solution 1
sampling_interval = 0.04
num_samples = 100

PH1 = 4.00
V1 = get_avg_volts(sampling_interval, num_samples)
print("V1 :",V1)

# Calibrated V1 : 1.589529

### Calibration Solution 2

Remove the PH sensor, clean it with fresh water and insert in second calibration solution.

#### Update variable "PH2" to the PH of reference solution 2

Wait for 30-60 seconds and run the cell below.
It is recommended to run the cell multiple times until V2 value stabilizes.

#### IMPORTANT : Run this cell only while calibrating the sensor

In [ ]:
# IMPORTANT : Run this cell only while calibrating the sensor

# Calibration solution 2
sampling_interval = 0.04
num_samples = 100

PH2 = 7.00
V2 = get_avg_volts(sampling_interval, num_samples)
print("V2 :",V2)

# Calibrated V2 : 1.45238

### Finish Calibration

Here we calculate values of K and OFFSET that are later used for PH measurement of unknown solutions.

#### IMPORTANT : Run this cell only while calibrating the sensor

In [ ]:
# IMPORTANT : Run this cell only while calibrating the sensor

# default values
OFFSET = 41.02740741
K = -19.18518519

# Calibrated values
# OFFSET = 38.769407615
# K = -21.87402348

if not (V2 - V1) :
    print("Calibration error!")
else :
    K = (PH2-PH1)/(V2-V1)
    OFFSET =( (PH2+PH1) - K*(V1+V2) )/2
    print("Offset :", OFFSET, " K :", K)

### Measure PH of Unknown Solution

Remove the PH sensor, clean it with fresh water and insert in unknown solution. Wait for 30 to 60 seconds and then run this cell.

#### IMPORTANT : Run this cell after calibration is complete to get correct results

In [ ]:
# Unknown solution
sampling_interval = 0.02
num_samples = 40

volts = get_avg_volts(sampling_interval, num_samples)
ph = round(K * volts + OFFSET,3) ;
print("V :", volts, "PH :", ph)